In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Naitik\\OneDrive\\Desktop\\Masters\\MLops\\car-pred-MLoperations\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Naitik\\OneDrive\\Desktop\\Masters\\MLops\\car-pred-MLoperations'

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
from carPred.constants import *
from carPred.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [24]:
import os
import urllib.request as request
import zipfile
from carPred import logger
from carPred.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-22 11:25:07,044: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 11:25:07,046: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 11:25:07,048: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-22 11:25:07,049: INFO: common: created directory at: artifacts]
[2024-02-22 11:25:07,051: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-22 11:25:08,020: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 14085
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "73de7097fdb6ffbe4607db738b0afb6b8a2687fc374931d3c437d9a21b65eb02"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B28A:2D523F:8AB7F5:A78734:65D7755A
Accept-Ranges: bytes
Date: Thu, 22